<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/%E8%A8%AD%E5%82%99%E6%83%85%E5%A0%B1%E5%BC%95%E7%B6%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 0) インストール & インポート（既に入っていれば pip はスキップされます）
try:
    import openstudio as osd
except ModuleNotFoundError:
    # Colab では先頭に '!' が必要
    !pip -q install openstudio
    import openstudio as osd

from google.colab import files

# ---- ユーティリティ ----
def load_model(path: str) -> osd.model.Model:
    """OSM を読み込んで Model を返す。失敗時は RuntimeError。"""
    opt_m = osd.model.Model.load(osd.path(path))
    if opt_m.is_initialized():
        return opt_m.get()
    raise RuntimeError(f"OpenStudio Model の読み込みに失敗: {path}")

def pick_uploaded_osm(upload_result: dict, label: str) -> str:
    """アップロード結果から最初の .osm を選ぶ。"""
    assert upload_result, f"{label}: ファイルがアップロードされていません。"
    cands = [n for n in upload_result.keys() if n.lower().endswith(".osm")]
    assert cands, f"{label}: 拡張子 .osm のファイルが見つかりません。"
    return cands[0]  # ラベルに依らず、最初の .osm を採用

# 1) 供与側（HVACあり）をアップロード
print("引き継ぐモデルデータを選択してください（HVACあり）")
uploaded_src = files.upload()
src = pick_uploaded_osm(uploaded_src, "Source")
print("Source OSM :", src)

# 2) 受け側（gbXML取込後）をアップロード
print("システムを付加するモデルを選択してください（gbXML取込後）")
uploaded_dst = files.upload()
dst = pick_uploaded_osm(uploaded_dst, "Target")
print("Target OSM :", dst)

# 3) モデル読み込み
src_m = load_model(src)
dst_m = load_model(dst)

# 4) ループ（Plant/Air）の複製のみ
for pl in src_m.getPlantLoops():
    _ = pl.clone(dst_m)
    print("[Clone] PlantLoop:", pl.nameString())

for al in src_m.getAirLoopHVACs():
    _ = al.clone(dst_m)
    print("[Clone] AirLoopHVAC:", al.nameString())

# 5) VRF 親機を clone & マップ化（重複 clone を避ける）
vrf_map = {}  # key: src_handle(str) -> value: dst_vrf
for vrf_src in src_m.getAirConditionerVariableRefrigerantFlows():
    opt_vrf_dst_obj = vrf_src.clone(dst_m).to_AirConditionerVariableRefrigerantFlow()
    if not opt_vrf_dst_obj.is_initialized():
        print("[Warn] VRF の型変換に失敗:", vrf_src.nameString())
        continue
    vrf_dst = opt_vrf_dst_obj.get()
    vrf_map[str(vrf_src.handle())] = vrf_dst
    print("[Clone] VRF (outdoor):", vrf_src.nameString(), "->", vrf_dst.nameString())

# 6) 端末は “親 VRF から辿って” clone & 接続（端末→親の逆参照を使わない）
processed = 0
for vrf_src in src_m.getAirConditionerVariableRefrigerantFlows():
    parent_dst = vrf_map.get(str(vrf_src.handle()))
    if parent_dst is None:
        print("[Warn] 対応する複製先 VRF が見つかりません:", vrf_src.nameString())
        continue

    # terminals() が使える環境では直接たどる
    terms = []
    if hasattr(vrf_src, "terminals"):
        try:
            terms = list(vrf_src.terminals())
        except Exception as e:
            print("[Warn] terminals() 取得に失敗:", vrf_src.nameString(), "-", e)

    # 一部バインディングで terminals() が未公開のことがあるためフォールバック
    if not terms:
        # 全端末から “親がこの vrf_src であるもの” をフィルター
        for t in src_m.getZoneHVACTerminalUnitVariableRefrigerantFlows():
            # ここでは handle API を使わないように try に包む
            same_parent = False
            try:
                # 端末 -> 親 VRF Optional を直接取れる環境なら使う
                if hasattr(t, "airConditionerVariableRefrigerantFlow"):
                    opt_parent = t.airConditionerVariableRefrigerantFlow()
                    if opt_parent.is_initialized() and opt_parent.get().handle() == vrf_src.handle():
                        same_parent = True
            except Exception:
                pass

            if not same_parent:
                # それでもダメならハンドル経由（失敗しても無視）
                try:
                    if hasattr(t, "airConditionerVariableRefrigerantFlowHandle"):
                        h = t.airConditionerVariableRefrigerantFlowHandle()
                        if (not h.isNull()) and (h == vrf_src.handle()):
                            same_parent = True
                except Exception:
                    pass

            if same_parent:
                terms.append(t)

    # 端末を clone → 親へ addTerminal
    for term_src in terms:
        opt_term_dst = term_src.clone(dst_m).to_ZoneHVACTerminalUnitVariableRefrigerantFlow()
        if not opt_term_dst.is_initialized():
            print("[Warn] VRF 端末の型変換に失敗:", term_src.nameString())
            continue
        term_dst = opt_term_dst.get()

        ok = False
        try:
            ok = parent_dst.addTerminal(term_dst)
        except Exception as e:
            print("[Warn] addTerminal で例外:", term_dst.nameString(), "-", e)

        if ok:
            processed += 1
            print("[Link ]", term_dst.nameString(), "->", parent_dst.nameString())
        else:
            print("[Warn] 端末の親接続に失敗:", term_dst.nameString())

print(f"[Info ] VRF 端末の接続完了: {processed} 台")


# 7) その他ゾーン機器（複製のみ）
for fc in src_m.getZoneHVACFourPipeFanCoils():
    _ = fc.clone(dst_m)
    print("[Clone] FanCoil:", fc.nameString())

for ptac in src_m.getZoneHVACPackagedTerminalAirConditioners():
    _ = ptac.clone(dst_m)
    print("[Clone] PTAC:", ptac.nameString())

for pthp in src_m.getZoneHVACPackagedTerminalHeatPumps():
    _ = pthp.clone(dst_m)
    print("[Clone] PTHP:", pthp.nameString())

# 8) 保存 & ダウンロード
OUT = "target_with_HVAC.osm"
dst_m.save(osd.path(OUT), True)
print("Saved:", OUT)

files.download(OUT)
# ======================================================================


引き継ぐモデルデータを選択してください（HVACあり）


Saving 下妻_空調.osm to 下妻_空調 (3).osm
Source OSM : 下妻_空調 (3).osm
システムを付加するモデルを選択してください（gbXML取込後）


Saving 90度回転.osm to 90度回転 (3).osm
Target OSM : 90度回転 (3).osm
[Clone] VRF (outdoor): 〇EHP-2-2 -> 〇EHP-2-2
[Clone] VRF (outdoor): 〇EHP-2-1 -> 〇EHP-2-1
[Clone] VRF (outdoor): 〇EHP-1-4-1 -> 〇EHP-1-4-1
[Clone] VRF (outdoor): 〇EHP-1-3-W -> 〇EHP-1-3-W
[Clone] VRF (outdoor): 〇EHP-1-4-2 -> 〇EHP-1-4-2
[Clone] VRF (outdoor): 〇EHP-1-3-M -> 〇EHP-1-3-M
[Clone] VRF (outdoor): EHP-2-3 -> EHP-2-3
[Clone] VRF (outdoor): 〇EHP-1-2 -> 〇EHP-1-2
[Clone] VRF (outdoor): EHP-1-1 -> EHP-1-1
[Link ] EHP-2-2-1 -> 〇EHP-2-2
[Link ] EHP-2-2-2 -> 〇EHP-2-2
[Link ] EHP-2-1-1 -> 〇EHP-2-1
[Link ] EHP-2-1-2 -> 〇EHP-2-1
[Link ] EHP-1-4-1 -> 〇EHP-1-4-1
[Link ] EHP-1-3-1-W -> 〇EHP-1-3-W
[Link ] EHP-1-4-2 1 -> 〇EHP-1-4-2
[Link ] EHP-1-3-1-M -> 〇EHP-1-3-M
[Link ] EHP-2-3-1 -> EHP-2-3
[Link ] VRF Zone Terminal -> EHP-1-1
[Link ] VRF Zone Terminal 1 -> EHP-1-1
[Info ] VRF 端末の接続完了: 11 台
Saved: target_with_HVAC.osm


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>